## 보스턴 주택 가격 예측하기

- 회귀(regression): 연속적인 값의 예측
- 보스턴 주택 가격 데이터셋 -> 1970년대 보스턴 지역의 범죄율, 토지 지역의 비율, 방의 개수 등 총 14개의 변수가 포함됨.

### 패키지 참조 및 데이터 준비

In [1]:
from tensorflow.keras.datasets.boston_housing import load_data
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from pandas import DataFrame

import numpy as np

- KFold는 하나의 데이터셋을 내가 원하는 등분으로 나누어 학습데이터, 검증데이터를 계속해서 바꿔가며 분석을 진행하는 방법이다.
- 다시 말해, 1-5까지 5등분을 했으면, 첫 번째에는 1-4까지가 학습데이터, 5가 검증데이터, 두 번째에는 1-3 + 5가 학습데이터, 4가 검증데이터 등등이 된다.
> KFold가 필요한 이유? 만약 보스턴 집값 데이터가 집값이 가장 저렴한 집 ~ 가장 비싼 집 순으로 입력되어 있다면 그 데이터셋 그대로 7:3으로 나누었을 때 정확한 분석 결과를 도출하기 어렵기 때문에

In [2]:
# 데이터 다운받기
(x_train, y_train), (x_test, y_test) = load_data(path = 'boston_housing.npz', 
                                                test_split = 0.33, seed = 777)

In [3]:
# 데이터 크기 확인하기
# 학습데이터와 학습 label의 크기
print(x_train.shape, y_train.shape)

(339, 13) (339,)


In [4]:
# 검증 데이터와 검증 label의 크기
print(x_test.shape, y_test.shape)

(167, 13) (167,)


### label 데이터 확인

In [5]:
# 주택 가격의 중간 가격($1,000 단위)를 의미함.
DataFrame(y_train)

,0
0,22.5
1,8.3
2,17.2
3,25.0
4,28.5
...,...
334,21.9
335,21.8
336,34.9
337,25.0


### 학습 데이터 확인
- 총 13개의 특성을 갖는 데이터셋
- 각 특성마다 데이터의 범위(스케일)가 다르다.
    - 범죄율 : 0 ~ 1
    - 방의 개수 : 3 ~ 9

In [6]:
DataFrame(x_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.20608,22.0,5.86,0.0,0.4310,5.593,76.5,7.9549,7.0,330.0,19.1,372.49,12.50
1,0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
2,3.67822,0.0,18.10,0.0,0.7700,5.362,96.2,2.1036,24.0,666.0,20.2,380.79,10.19
3,0.63796,0.0,8.14,0.0,0.5380,6.096,84.5,4.4619,4.0,307.0,21.0,380.02,10.26
4,0.04294,28.0,15.04,0.0,0.4640,6.249,77.3,3.6150,4.0,270.0,18.2,396.90,10.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,0.07151,0.0,4.49,0.0,0.4490,6.121,56.8,3.7476,3.0,247.0,18.5,395.15,8.44
163,12.04820,0.0,18.10,0.0,0.6140,5.648,87.6,1.9512,24.0,666.0,20.2,291.55,14.10
164,0.10328,25.0,5.13,0.0,0.4530,5.927,47.2,6.9320,8.0,284.0,19.7,396.90,9.22
165,0.10000,34.0,6.09,0.0,0.4330,6.982,17.7,5.4917,7.0,329.0,16.1,390.43,4.86


### 데이터 표준화 수행
- 스케일이 서로 다를 경우 신경망의 성능에 큰 영향을 주기 때문에 표준화를 수행해야 한다.
- 표준화 -> 각 값에 대해 특성(DF의 컬럼)의 평균을 때고 표준편차로 나누는 처리
- 표준화는 특성의 평균을 0으로, 표준편차를 1로 만들어준다.

In [7]:
# 평균 구하기
mean = np.mean(x_train, axis = 0)

In [8]:
# 표준편차 구하기
std = np.std(x_train, axis = 0)

In [9]:
# 학습 데이터 표준화
std_x_train = (x_train - mean) / std

# 검증 데이터 표준화
std_x_test = (x_test - mean) / std

In [10]:
# 학습데이터 표준화 확인
DataFrame(std_x_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.398157,-0.493015,-0.114676,-0.311588,-0.572882,-0.714780,0.135436,0.258581,-0.637411,-0.789973,0.107780,0.384607,0.703446
1,1.352052,-0.493015,0.970928,-0.311588,0.994262,-0.554323,0.939305,-0.871380,1.641939,1.518963,0.834895,-3.607039,1.563816
2,0.403808,-0.493015,0.970928,-0.311588,0.317915,-0.950494,1.027837,-1.081587,1.641939,1.518963,0.834895,-0.397411,1.837015
3,-0.412229,0.556941,-0.903942,-0.311588,-0.869815,0.675368,-0.902158,1.934138,-0.181541,-0.748424,0.607672,0.448912,-0.460025
4,-0.422487,2.866843,-0.929962,-0.311588,-1.216236,0.815945,-1.451056,0.610605,-0.637411,-0.979912,0.380448,0.462715,-1.298649
...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,-0.421010,2.866843,-1.119328,-0.311588,-1.372951,-0.253290,-1.305864,2.506859,-0.979314,-0.564422,-0.892003,0.420785,-0.858270
335,-0.110380,-0.493015,0.970928,-0.311588,-0.218213,-0.744599,-1.011938,0.140008,1.641939,1.518963,0.834895,0.421099,-0.334980
336,-0.422647,2.656852,-1.219071,-0.311588,-1.076018,1.047399,-1.879550,0.742025,-0.751379,-0.938363,-0.028554,0.449331,-1.482140
337,0.216240,-0.493015,0.970928,-0.311588,-0.218213,1.099938,0.287710,-0.177775,1.641939,1.518963,0.834895,0.445776,-0.798465


In [11]:
# 검증 데이터 표준화 확인
DataFrame(std_x_test)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,-0.403305,0.430946,-0.798418,-0.311588,-1.051274,-0.984573,0.270004,1.922124,-0.295509,-0.475388,0.335004,0.207479,-0.052267
1,-0.424226,0.241954,-1.446021,-0.311588,-1.174171,1.160997,-0.332013,2.508060,-0.751379,-1.152043,0.107780,0.424445,-0.657109
2,-0.013963,-0.493015,0.970928,-0.311588,1.744841,-1.312586,0.967635,-0.781733,1.641939,1.518963,0.834895,0.294266,-0.366241
3,-0.354877,-0.493015,-0.468834,-0.311588,-0.168724,-0.270330,0.553306,0.308026,-0.637411,-0.611906,1.198453,0.286214,-0.356727
4,-0.421599,0.682936,0.528591,-0.311588,-0.779085,-0.053075,0.298334,-0.083323,-0.637411,-0.831522,-0.073998,0.462715,-0.311873
...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,-0.418395,-0.493015,-0.996457,-0.311588,-0.902807,-0.234831,-0.427627,-0.022049,-0.751379,-0.968040,0.062336,0.444416,-0.604100
163,0.924589,-0.493015,0.970928,-0.311588,0.458134,-0.906475,0.663086,-0.852157,1.641939,1.518963,0.834895,-0.638844,0.165204
164,-0.414832,0.556941,-0.903942,-0.311588,-0.869815,-0.510304,-0.767590,1.449447,-0.181541,-0.748424,0.607672,0.462715,-0.498083
165,-0.415200,0.934925,-0.765170,-0.311588,-1.034777,0.987761,-1.812266,0.783891,-0.295509,-0.481324,-1.028337,0.395063,-1.090692


### K-Fold

In [33]:
# 분할할 폴드 수
k = 3

In [34]:
# 주어진 데이터셋을 k만큼 등분
# 여기서는 3이므로 훈련 데이터셋(404개)를 3등분하여 
# 1개는 검증셋으로, 나머지는 훈련셋으로 활용
kfold = KFold(n_splits = k, random_state = 777)

# 흠.. 워닝에 shuffle을 True로 해야 random_state가 의미있다고 하는데.. 그럼 굳이 k-fold를 할 필요가 있나?

In [46]:
# 표준화한 데이터를 분할
split_x_train = kfold.split(std_x_train)
split_x_train

# 분할된 데이터 수만큼 반복 수행

# 데이터셋을 평가한 후 결과 mae를 담을 리스트
mae_list = []

# k번 진행 - 각각 훈련셋 인덱스와 검증셋 인덱스 세트가 달라진다
for train_index, val_index in split_x_train:
    print(f'훈련셋 인덱스: {train_index}')
    print(f'검증셋 인덱스: {val_index}')
    
    # 해당 인덱스는 무작위로 생성된다.
    # 무작위로 생성하는 것이 과대적합을 피할 수 있는 좋은 방법이다.
    x_train_fold, x_val_fold = std_x_train[train_index], std_x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # 모델 생성
    model = Sequential()
    
    # 13차원의 데이터를 입력으로 받고, 64개의 출력을 가지는 첫 번째 Dense층
    model.add(Dense(64, activation = 'relu', input_shape = (13, )))

    # 32개의 출력을 가지는 Dense층
    model.add(Dense(32, activation = 'relu'))

    # 하나의 값을 출력
    # -> 정답의 범위가 정해지지 않기 때문에 활성화 함수는 linear
    # -> linear는 기본값이므로 생략 가능함
    model.add(Dense(1, activation = 'linear'))

    model.compile(optimizer = 'adam', loss = 'mse', metrics = ['mae'])

    result = model.fit(x_train_fold, y_train_fold,
                      epochs = 300,
                      validation_data = (x_val_fold, y_val_fold))

    # 모델 평가
    tmp, test_mae = model.evaluate(std_x_test, y_test)

    # 평가 결과를 list에 추가함
    mae_list.append(test_mae)

훈련셋 인덱스: [113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148
 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166
 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184
 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202
 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220
 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238
 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256
 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274
 275 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292
 293 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310
 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328
 329 330 331 332 333 334 335 336 337 338]
검증셋 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15

### 결과 확인

In [47]:
print(f'전체 결과(오차 리스트): {mae_list}')
print(f'오차들의 평균값을 최종 결과로 사용: {np.mean(mae_list)}')

전체 결과(오차 리스트): [2.2741539478302, 2.0924527645111084, 2.330174446105957]
오차들의 평균값을 최종 결과로 사용: 2.2322603861490884


### 내용 정리

- 회귀는 분류에서 사용했던 것과는 다른 손실 함수를 사용한다.
    - 평균 제곱 오차(MSE)는 회귀에서 자주 사용되는 손실 함구
- 회귀에서 사용되는 평가 지표도 분류와 다르다.
    - 정확도 개념은 회귀에 적용되지 않는다.
    - 일반적인 회귀 지표는 평균 절대 오차(MAE)이다.
- 입력 데이터의 특성이 서로 다른 범위를 가지면 전처리 단계에서 각 특성을 개별적으로 스케일 조정해야 한다.
    - 표준화
- 가용한 데이터가 적다면 'K-폴드 검증'을 사용하는 것이 신뢰할 수 있는 모델 평가 방법이다.
- 가용한 훈련 데이터가 적다면 과대적합을 피하기 위해 은닉 층의 수를 줄인 모델이 좋다.(일반적으로 1개 또는 2개)
    - model.add()를 적게 하라는 뜻